In [ ]:
import openrouteservice as ors
api_key = '5b3ce3597851110001cf6248b7d7645b7a3141d59babc93a00bf83ae'

# Create a client with your API key
client = ors.Client(key=api_key)

# Coordinates of the starting and ending points
coordinate_1 = [54.99774379754072, -1.5024074642072465]
coordinate_2 = [55.021733703401985, -1.582868699692162]

coordinates_ORS = [list(reversed(coordinate_1)),list(reversed(coordinate_2))]

#reverse the list since ors takes lat then long coords
##coordinates_ORS = list(reversed(coordinates))




# Calculate the route
route = client.directions(coordinates = coordinates_ORS ,profile='driving-car',format = 'geojson')

if route and 'features' in route and len(route['features']) > 0:
    # Extract the duration from the first route found
    duration_seconds = route['features'][0]['properties']['segments'][0]['duration']
    duration_minutes = duration_seconds / 60
    print(f"Estimated travel time: {duration_minutes} minutes")
else:
    print("No route found.")


In [53]:
#Import out excel sheet into python using pandas
import pandas as pd
greggs = pd.read_csv(r"C:\Users\jconn\Documents\Oliver Wyman Data Challenge\greggs_loacation.csv")

# Lets create a column for our coordinates  
greggs[['Latitude', 'Longitude']] = greggs['Coordinates'].str.split(',', expand=True)


#delete original coordinates column
greggs = greggs.drop(columns=['Coordinates'])
greggs = greggs.drop(columns=['#'])
greggs



,Location,Latitude,Longitude
0,Airport-skyway,55.03765134510472,-1.7138647002471277
1,Airport,55.0380031124521,-1.709734289070828
2,Kingston Park,55.013085021374906,-1.6689824997506324
3,Kenton Lane,55.00148283394153,-1.6660893844102314
4,Eldon Sq,54.97514749896339,-1.6153630439276734
...,...,...,...
64,Whitley Bay,55.0447391386017,-1.444999989384653
65,Cobalt,55.020734283157694,-1.5016482457729423
66,Battle Hill,55.006555577549626,-1.5182861402857417
67,Saltmeadows Road,54.96764265593575,-1.5887707304334757


In [54]:
#Cretes a data frame with only the outlet stores
outlet_rows = greggs.loc[[9, 24, 37, 44, 57]]
outlets = pd.DataFrame(outlet_rows)

#Creates a new data frame with only our regular stores
regulars = greggs.drop([9, 24, 37, 44, 57])
regulars
outlets


,Location,Latitude,Longitude
9,Howdon,54.99774379754072,-1.5024074642072465
24,Old Durham Road,54.95129239645862,-1.595720601598768
37,Frederick St.,54.99090840840786,-1.437465492042934
44,Wellbeck Road,54.97575304267181,-1.5614199349310112
57,Arthur's Hill,54.97386109838361,-1.6386043517009117


In [1]:
####  THIS IS CODE I WOULD USE USING OPENROUTESERVICES API TO CALCULATE THE TRAVEL TIME BETWEEN EACH OUTLET AND EACH STORE
'''
import numpy as np

#we need to make a matrix that has the transport times for each location to each store
N_outlet = 5
N_regular = 64

#initialise our matrix
travel_time_matrix = np.zeros((64,5))

for i in range(N_regular-1):
    for j in range(N_outlet-1):
        
        coords_i_j = [
            [outlets.iloc[j]['Longitude'],outlets.iloc[j]['Latitude']],
            [regulars.iloc[i]['Longitude'],regulars.iloc[i]['Latitude']]
        ]
        
        
        route = client.directions(coordinates = coords_i_j ,profile='driving-car',format = 'geojson')
        duration_seconds = route['features'][0]['properties']['segments'][0]['duration']
        duration_minutes = duration_seconds/60
        
        
        
        travel_time_matrix[i][j] = duration_minutes
'''


"\nimport numpy as np\n\n#we need to make a matrix that has the transport times for each location to each store\nN_outlet = 5\nN_regular = 64\n\n#initialise our matrix\ntravel_time_matrix = np.zeros((64,5))\n\nfor i in range(N_regular-1):\n    for j in range(N_outlet-1):\n        \n        coords_i_j = [\n            [outlets.iloc[j]['Longitude'],outlets.iloc[j]['Latitude']],\n            [regulars.iloc[i]['Longitude'],regulars.iloc[i]['Latitude']]\n        ]\n        \n        \n        route = client.directions(coordinates = coords_i_j ,profile='driving-car',format = 'geojson')\n        duration_seconds = route['features'][0]['properties']['segments'][0]['duration']\n        duration_minutes = duration_seconds/60\n        \n        \n        \n        travel_time_matrix[i][j] = duration_minutes\n"

In [55]:
#GIVEN THE FINANCIAL RESOURCES I HAVE AVAILABLE A MAP API WHICH CAN DO IN EXCESS OF 320 REQUESTS PER DAY IS TOO EXPENSIVE
#THE CODE USING THE OPENRESCOURCESSERVICE API IS ABOVE
#I AM GOING TO ASSUME THAT THERE ARE DIRECT, IDENTICAL, TRAFFICLESS ROADS BETWEEN EACH GREGGS STORE
#I.E THE DIRECT DISTANCE BETWEEN STORES IS PROPORTIONATE TO THE TIME IT TAKE FOR THE LORRY TO GET TO EACH STORE FROM A GIVEN STORE

#I want to create a matrix that calculates the distance
#it is much more practical to think of these in KMs so we are going to use the Haversine formula to convert


def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = lat1 * np.pi / 180
    lon1 = lon1 * np.pi / 180
    lat2 = lat2 * np.pi / 180
    lon2 = lon2 * np.pi / 180
    
    # Radius of the Earth in kilometers
    earth_radius = 6371  # Average radius
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Calculate the distance
    distance = earth_radius * c
    
    return distance


In [56]:
#Lets initialise our 64x 5 matrix
import numpy as np

N_outlet = 5
N_regular = 64

#Initialise our 64x5 matrix which is going to store distances between i-th regular store and j-th outlet
distance_matrix = np.zeros((N_regular,N_outlet))

for i in range(N_regular):
    for j in range(N_outlet):
        
        distance_matrix[i][j] = haversine(float(regulars.iloc[i]['Latitude']),float(regulars.iloc[i]['Longitude']),float(outlets.iloc[j]['Latitude']),float(outlets.iloc[j]['Longitude']))
        
print(distance_matrix)


[[14.19211874 12.20674027 18.37263764 11.91103103  8.56424579]
 [13.95469206 12.07705011 18.13139523 11.72008355  8.45243711]
 [10.75861792  8.31042398 14.96963346  8.01919008  4.77260391]
 [10.44780873  7.16329943 14.63005837  7.2646797   3.53669946]
 [ 7.63205422  2.93401128 11.48538201  3.44318952  1.49012568]
 [ 7.59944934  2.82165584 11.43318456  3.37867778  1.5517544 ]
 [ 7.55675047  2.69696463 11.36815278  3.30228891  1.63449009]
 [ 4.92896316  4.14646678  8.9375719   1.61960633  4.172395  ]
 [ 2.2801784   5.89000544  6.33416975  2.39925595  6.80638902]
 [ 3.75508919 11.45881943  2.08945106  8.18657598 12.83904479]
 [ 7.60125616  9.87999803 11.4912265   7.49073886  7.85663365]
 [ 5.66318725  7.7392859   9.77932574  5.13815341  6.3167577 ]
 [ 5.78252682  7.87551673  9.88543473  5.29267088  6.40109419]
 [ 6.44947236  6.09049709 10.65921723  4.25819876  4.24392355]
 [ 6.51784823  4.61377098 10.63651441  3.30319414  3.01992052]
 [ 7.40028673  3.14560213 11.29818836  3.30290033  1.68

In [134]:
# RECALL THE J INDEXES AS:
#1 = HOWDON , 2 =OLD DURHAM ROAD, 3 = FREDERICK ST, 4= WELLBECK RD, 5 = ARTHURS HILL

#Initialise lists to store the regular stores to their respective nearest outlet
closest_to_Howdon = []
closest_to_OldDurhamRd = []
closest_to_FrederickSt = []
closest_to_WellbeckRd = []
closest_to_ArthursHill = []

indexes = []

for i in range(N_regular):
    if np.argmin(distance_matrix[i]) == 0:
        closest_to_Howdon.append(regulars.iloc[i]['Location'])
    
    elif np.argmin(distance_matrix[i]) == 1:
        closest_to_OldDurhamRd.append(regulars.iloc[i]['Location'])
    
    elif np.argmin(distance_matrix[i]) == 2:
        closest_to_FrederickSt.append(regulars.iloc[i]['Location'])
        
        #lets also create rows for a new df with just the frederick st stores in we can use in the next problem
        indexes.append(i)
    
    elif np.argmin(distance_matrix[i]) == 3:
        closest_to_WellbeckRd.append(regulars.iloc[i]['Location'])
    
    else:
        closest_to_ArthursHill.append(regulars.iloc[i]['Location'])
        
#our lists now tell us which regular stores have their closest outlet as the one after the list name 
#eg closest_to_Howdon tells us all the rgular stores whose closest outlet store is Howdon



    

print(regulars)



             Location            Latitude             Longitude
0      Airport-skyway   55.03765134510472   -1.7138647002471277
1             Airport    55.0380031124521    -1.709734289070828
2       Kingston Park  55.013085021374906   -1.6689824997506324
3         Kenton Lane   55.00148283394153   -1.6660893844102314
4            Eldon Sq   54.97514749896339   -1.6153630439276734
..                ...                 ...                   ...
64        Whitley Bay    55.0447391386017    -1.444999989384653
65             Cobalt  55.020734283157694   -1.5016482457729423
66        Battle Hill  55.006555577549626   -1.5182861402857417
67   Saltmeadows Road   54.96764265593575   -1.5887707304334757
68  Team Valley South   54.92709258285387     -1.61730861678117

[64 rows x 3 columns]


In [95]:
# We now want to add 2 colums to the outlets df which tell us the nearest 
# we need to find the arg min and arg max in each column and then 

#Initialise lists where we are going to store the fursrthest and closest reg. store from each outlet
closest_stores = []
furthest_stores = []

for j in range(N_outlet):
    closest_stores.append(regulars.iloc[np.argmin(distance_matrix[:,j])]['Location'])
    furthest_stores.append(regulars.iloc[np.argmax(distance_matrix[:,j])]['Location'])


    
#Add the two columns
outlets['Closest Store'] = closest_stores
outlets['Furthest Store'] = furthest_stores

outlets.head()



,Location,Latitude,Longitude,Closest Store,Furthest Store
9,Howdon,54.99774379754072,-1.5024074642072465,Battle Hill,Airport-skyway
24,Old Durham Road,54.95129239645862,-1.595720601598768,Trinity Square,Whitley Bay
37,Frederick St.,54.99090840840786,-1.437465492042934,South Shields Interchange,Airport-skyway
44,Wellbeck Road,54.97575304267181,-1.5614199349310112,Shields Road East,Airport-skyway
57,Arthur's Hill,54.97386109838361,-1.6386043517009117,West Road,Prince Edward Road


In [119]:
# Find the shortest route for the Frederick St. Lorry to take
#It starts and finishes at the outlet store
#We will assume there are direct roads running to each store
#i.e each store is a node and we want to find the shortest path such that we visit each node once.
#There are 5040 differnent combinations 

# Fistly lets use Folium to plot these points on a map

import folium

FrederickSt_lat = float(outlets.iloc[2]['Latitude'])
FrederickSt_lon = float(outlets.iloc[2]['Longitude'])

#initialise our map
greggs_map = folium.Map(location=[FrederickSt_lat,FrederickSt_lon],zoom_start = 13,tiles = "cartodbpositron")

folium.Marker([FrederickSt_lat,FrederickSt_lon], popup = 'Frederick St.').add_to(greggs_map)

for i in range(len(closest_to_FrederickSt)):
    folium.Marker([float(),float()], popup = 'Frederick St.').add_to(greggs_map)
    
    
    

mapit.save( 'map.html')


m = folium.Map(location=list(reversed([-77.0362619,38.897475 ])), zoom_start = 13,tiles = "cartodbpositron")

greggs_map



7